In [ ]:
using LinearAlgebra, Test

In [9]:
# We want to implement QR with householder

function householderreflection(x)
    T = float(eltype(x)) # float(Int) returns Float64
    y = Vector{T}(x)
    y[1] += norm(x)*sign(x[1])
    w = y/norm(y)
    I - 2w*w'
end

householderreflection (generic function with 1 method)

In [34]:
a = randn(5)
Q = householderreflection(a)


A = [1 2 3;
     4 5 6;
     7 8 9;
     10 11 13]


m,n = size(A)
𝐚₁ = A[:,1]
Q₁ = householderreflection(𝐚₁)

Q₁A = Q₁*A
α₁ = Q₁A[1,1]
𝐰₁ = Q₁A[1,2:end]

A₂ = Q₁A[2:end,2:end]

𝐚₂ = A₂[:,1]
Q₂ = householderreflection(𝐚₂)
Q₂A₂ = Q₂*A₂
α₂ = Q₂A₂[1,1]
𝐰₂ = Q₂A₂[1,2:end]

A₃ = Q₂A₂[2:end,2:end]
𝐚₃ = A₃[:,1]
Q₃ = householderreflection(𝐚₃)

Q̃₂ = zeros(m,m)
Q̃₂[1,1] = 1
Q̃₂[2:end,2:end] = Q₂

Q̃₃ = zeros(m,m)
Q̃₃[1,1] = 1
Q̃₃[2,2] = 1
Q̃₃[3:end,3:end] = Q₃


R = Q̃₃ * Q̃₂ *Q₁ * A
Q =  Q₁ * Q̃₂ *Q̃₃

@test Q*R ≈ A

Test Passed

In [55]:
function householderqr(A)
    m,n = size(A)

    T = float(eltype(A))
    R = zeros(T, m, n) # right-triangular
    Q = Matrix{T}(I, m, m) # identity matrix

    Aⱼ = A

    for j = 1:n
        𝐚ⱼ = Aⱼ[:,1]
        Qⱼ = householderreflection(𝐚ⱼ)
        QⱼAⱼ = Qⱼ*Aⱼ

        α = QⱼAⱼ[1,1]
        𝐰 = QⱼAⱼ[1,2:end]
        Aⱼ₊₁ = QⱼAⱼ[2:end,2:end] # drop first row and col


        # populate R
        R[j,j] = α
        R[j,j+1:end] = 𝐰

        # update Q
        Q[:,j:end] = Q[:,j:end]*Qⱼ
        Aⱼ = Aⱼ₊₁
    end
    Q,R
end

Q * R

4×3 Matrix{Float64}:
  1.0   2.0   3.0
  4.0   5.0   6.0
  7.0   8.0   9.0
 10.0  11.0  13.0

In [58]:
A = randn(10,10)
Q,R = householderqr(A)
inv(A) ≈ inv(R) * Q'

true

In [59]:
b = randn(10)
A\b # A \ b is code for inv(A) * b but doesn't have to form the inv(A)

10-element Vector{Float64}:
  1.229658513452891
 -0.34081981687625956
  5.239835293957892
  2.4706356133706535
  0.44221660606629115
 -1.8107548328267402
 -0.16271477290410102
 -3.279547413680206
  1.6351204658543932
  0.5237014892089888

In [62]:
UpperTriangular(R)\(Q'b) # R \ is backsubs

10-element Vector{Float64}:
  1.2296585134528912
 -0.34081981687625834
  5.239835293957885
  2.4706356133706504
  0.44221660606629315
 -1.810754832826739
 -0.16271477290409944
 -3.2795474136802016
  1.6351204658543932
  0.5237014892089881

In [61]:
# x/y is code for x*inv(y)
#x \ y is code for inv(x)*y

10×10 Matrix{Float64}:
 -3.59207  -1.13372  0.911697  -0.0562461  …   0.836298   1.51439   -1.19078
  0.0       2.58082  0.642188  -0.975118      -0.513718   0.432444   0.276725
  0.0       0.0      1.68178   -0.462981       1.46599    0.293676  -1.14696
  0.0       0.0      0.0        2.29619        0.815966   0.152544  -1.83037
  0.0       0.0      0.0        0.0            0.424796   0.654596   0.992234
  0.0       0.0      0.0        0.0        …   0.331271  -1.9053     0.518941
  0.0       0.0      0.0        0.0           -0.695709  -2.14948    1.2276
  0.0       0.0      0.0        0.0            0.954703   0.845727   1.61138
  0.0       0.0      0.0        0.0            0.0       -2.06696    2.20017
  0.0       0.0      0.0        0.0            0.0        0.0        0.96942